In [1]:
!pip install xlstm torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00


In [2]:
import os
import re
import csv
import json

import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

from torch_geometric.data import Batch
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

# xLSTM
from xlstm import (
    xLSTMBlockStack, xLSTMBlockStackConfig,
    mLSTMBlockConfig, mLSTMLayerConfig,
    sLSTMBlockConfig, sLSTMLayerConfig,
    FeedForwardConfig
)

# ======================================================
# Dataset
# ======================================================


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Để kết quả ổn định hơn trên GPU (có thể chậm hơn chút)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def normalize_name(name):
    """
    Chuẩn hoá tên file:
    - Bỏ đuôi _seq.pt hoặc .pt
    - Bỏ phần .exe, .EXE, .ex, .EX...
    """
    name = name.replace("_seq.pt", "")
    name = name.replace(".pt", "")
    name = re.sub(r"\.(e?xe?)$", "", name, flags=re.IGNORECASE)
    return name


class MultiModalDatasetPT(Dataset):
    """
    Dataset đọc graph .pt + seq_ids .pt
    - Hỗ trợ:
        + 1 thư mục benign graph + benign seq (benign_seq_root)
        + nhiều thư mục ransomware graph + ransomware seq (ransomware_seq_root)
    - Cho phép:
        + CLEAN: benign_seq_root = ransomware_seq_root = seq_ids clean
        + NOISE: benign_seq_root = seq_ids clean, ransomware_seq_root = seq_ids_noise
    - Tự xử lý graph rỗng bằng cách tạo dummy node
    - Gắn thêm cờ has_graph cho mỗi sample
    """
    def __init__(
        self,
        benign_graph_dir,
        ransomware_graph_dirs,
        benign_seq_root,
        ransomware_seq_root,
        max_seq_len=1500
    ):
        self.max_len = max_seq_len
        self.samples = []

        mapping = []

        # Benign
        mapping.append((benign_graph_dir, os.path.join(benign_seq_root, "benign"), 0))

        # Ransomware
        for r_dir in ransomware_graph_dirs:
            mapping.append((r_dir, os.path.join(ransomware_seq_root, "ransomware"), 1))

        # Gom tất cả cặp (graph, seq, label)
        for gdir, sdir, label in mapping:
            if not (os.path.isdir(gdir) and os.path.isdir(sdir)):
                print(f"[WARN] Thiếu thư mục: {gdir} hoặc {sdir}")
                continue

            graph_map = {}
            for f in sorted(os.listdir(gdir)):
                if f.endswith(".pt"):
                    norm = normalize_name(f)
                    graph_map[norm] = os.path.join(gdir, f)

            for fname in sorted(os.listdir(sdir)):
                if not fname.endswith("_seq.pt"):
                    continue
                norm = normalize_name(fname)
                spath = os.path.join(sdir, fname)

                if norm in graph_map:
                    ppath = graph_map[norm]
                    self.samples.append((ppath, spath, label))

        if not self.samples:
            raise RuntimeError("Không tìm thấy cặp graph + seq nào. Kiểm tra tên file & thư mục.")

        self.samples.sort(key=lambda x: x[0])

        print(f"[INFO] Tổng số sample: {len(self.samples)}")

        # Suy ra feat_dim từ một graph hợp lệ
        self.feat_dim = None
        for g_path, s_path, lbl in self.samples:
            data = torch.load(g_path, weights_only=False)
            if hasattr(data, "x") and data.x is not None and data.x.dim() == 2 and data.x.size(0) > 0:
                self.feat_dim = data.x.size(1)
                break

        if self.feat_dim is None:
            raise RuntimeError("Không tìm được graph hợp lệ để suy ra feat_dim!")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, i):
        ppath, spath, label = self.samples[i]

        data = torch.load(ppath, weights_only=False)
        seq_ids = torch.load(spath)

        # Xử lý graph rỗng: tạo dummy node + cờ has_graph
        has_real_graph = True
        if (not hasattr(data, "x")) or (data.x is None) or (data.x.dim() != 2) or (data.x.size(0) == 0):
            has_real_graph = False
            data.x = torch.zeros((1, self.feat_dim), dtype=torch.float32)
            data.edge_index = torch.empty((2, 0), dtype=torch.long)

        data.has_graph = torch.tensor(1 if has_real_graph else 0, dtype=torch.long)

        # Pad/cắt sequence
        if self.max_len is not None:
            if seq_ids.size(0) > self.max_len:
                seq_ids = seq_ids[:self.max_len]
            else:
                pad_len = self.max_len - seq_ids.size(0)
                pad = torch.zeros(pad_len, dtype=seq_ids.dtype)
                seq_ids = torch.cat([seq_ids, pad], dim=0)

        return data, seq_ids, torch.tensor(int(label), dtype=torch.float32)


def collate_fn(batch):
    graphs, seqs, labels = zip(*batch)
    return Batch.from_data_list(graphs), torch.stack(seqs), torch.stack(labels)


# ======================================================
# Encoders & Fusion classifier với gating w_seq / w_graph
# ======================================================

class GCNEncoder(nn.Module):
    def __init__(self, in_feats, hidden=64, drop=0.3):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden)
        self.bn1 = BatchNorm(hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.bn2 = BatchNorm(hidden)
        self.drop = drop
        self.output_dim = hidden

    def forward(self, x, edge_index, batch):
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, self.drop, training=self.training)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = F.dropout(x, self.drop, training=self.training)
        return global_mean_pool(x, batch)


class xLSTMEncoder(nn.Module):
    def __init__(self, vocab_size, embed=128, seq_len=1500, blocks=1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed, padding_idx=0)

        cfg = xLSTMBlockStackConfig(
            mlstm_block=mLSTMBlockConfig(
                mlstm=mLSTMLayerConfig(
                    conv1d_kernel_size=4,
                    qkv_proj_blocksize=4,
                    num_heads=4
                )
            ),
            slstm_block=sLSTMBlockConfig(
                slstm=sLSTMLayerConfig(
                    backend="vanilla",
                    num_heads=4,
                    conv1d_kernel_size=4,
                    bias_init="powerlaw_blockdependent"
                ),
                feedforward=FeedForwardConfig(
                    proj_factor=1.3,
                    act_fn="gelu"
                )
            ),
            context_length=seq_len,
            num_blocks=blocks,
            embedding_dim=embed,
            slstm_at=[0]
        )

        self.xlstm = xLSTMBlockStack(cfg)
        self.output_dim = embed

    def forward(self, seq):
        out = self.xlstm(self.embed(seq))  # [B, L, D]
        return out.mean(dim=1)             # [B, D]


class MLPClassifier(nn.Module):
    def __init__(self, in_dim, hiddens=[128, 64], drop=0.3):
        super().__init__()
        layers = []
        dims = [in_dim] + hiddens
        for i in range(len(hiddens)):
            layers.extend([
                nn.Linear(dims[i], dims[i + 1]),
                nn.ReLU(),
                nn.Dropout(drop)
            ])
        layers.append(nn.Linear(dims[-1], 1))
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        out = self.mlp(x)   # [B, 1]
        return out.view(-1) # [B]


class MultiModalClassifier(nn.Module):
    """
    F: encoder chung cho cả source/target
       - GCNEncoder + xLSTMEncoder
       - Gating w_seq / w_graph
    """
    def __init__(self, graph_enc, seq_enc, fusion_h=128):
        super().__init__()
        self.graph_enc = graph_enc
        self.seq_enc = seq_enc

        d_g = graph_enc.output_dim
        d_s = seq_enc.output_dim

        # Gate: cho ra 2 logit (seq, graph)
        self.gate = nn.Linear(d_s + d_g, 2)

        fusion_dim = d_g + d_s
        self.fusion_dim = fusion_dim   # dùng cho DomainDiscriminator
        self.classifier = MLPClassifier(fusion_dim, [fusion_h, fusion_h // 2])

    def forward(self, g, seq, return_branch=False, return_features=False):
        h_g = self.graph_enc(g.x, g.edge_index, g.batch)
        h_s = self.seq_enc(seq)

        # Nếu batch graph và batch seq lệch thì cắt theo min
        if h_g.size(0) != h_s.size(0):
            b = min(h_g.size(0), h_s.size(0))
            h_g = h_g[:b]
            h_s = h_s[:b]
            if hasattr(g, "has_graph"):
                g.has_graph = g.has_graph[:b]

        B = h_s.size(0)
        if hasattr(g, "has_graph"):
            has_graph = g.has_graph.view(-1).to(h_g.device).float()
        else:
            has_graph = torch.ones(B, device=h_g.device)

        # ép h_g = 0 nếu không có graph
        h_g = h_g * has_graph.view(-1, 1)

        # logit branch
        gate_in = torch.cat([h_s, h_g], dim=-1)      # [B, d_s + d_g]
        scores = self.gate(gate_in)                  # [B, 2]

        # ép logit graph nhỏ nếu has_graph=0
        scores[:, 1] = scores[:, 1] + (has_graph - 1.0) * 1e4

        w = torch.softmax(scores, dim=-1)            # [B, 2]
        w_seq = w[:, 0:1]
        w_graph = w[:, 1:2]

        h_fused = torch.cat([w_seq * h_s, w_graph * h_g], dim=-1)
        logits = self.classifier(h_fused)

        if not return_branch and not return_features:
            return logits

        branch_info = {
            "w_seq": w_seq.detach().cpu(),
            "w_graph": w_graph.detach().cpu(),
            "has_graph": has_graph.detach().cpu()
        }

        if return_branch and not return_features:
            return logits, branch_info
        if return_features and not return_branch:
            return logits, h_fused
        if return_branch and return_features:
            return logits, h_fused, branch_info

        return logits


# ======================================================
# Adversarial DA: GRL + Domain Discriminator
# ======================================================

class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambd):
        ctx.lambd = lambd
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambd * grad_output, None


def grad_reverse(x, lambd=1.0):
    return GradReverse.apply(x, lambd)


class DomainDiscriminator(nn.Module):
    """
    D: phân biệt feature thuộc domain source (0) hay target (1).
    """
    def __init__(self, in_dim, hidden=128, drop=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden, hidden // 2),
            nn.ReLU(),
            nn.Dropout(drop),
            nn.Linear(hidden // 2, 1),
        )

    def forward(self, feat, lambd=1.0):
        feat_rev = grad_reverse(feat, lambd)
        out = self.net(feat_rev)
        return out.squeeze(1)


# ======================================================
# Dataset wrapper để thêm domain label
# ======================================================

class DomainSubset(Dataset):
    """
    Wrapper: (ds, indices, domain_labels) -> (graph, seq, label, domain_id)
    domain_id: 0 = source, 1 = target
    """
    def __init__(self, base_ds, indices, domain_labels):
        self.base_ds = base_ds
        self.indices = sorted(indices)
        self.domain_labels = domain_labels

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        g, seq, y = self.base_ds[real_idx]
        d = self.domain_labels[real_idx]
        return g, seq, y, torch.tensor(float(d), dtype=torch.float32)


def collate_fn_da(batch):
    graphs, seqs, labels, domains = zip(*batch)
    return (
        Batch.from_data_list(graphs),
        torch.stack(seqs),
        torch.stack(labels),
        torch.stack(domains),
    )


# ======================================================
# Train / Eval
# ======================================================

def train_epoch_da(
    model,
    domain_disc,
    loader,
    cls_crit,
    dom_crit,
    opt_main,
    opt_disc,
    device,
    lambda_da=1.0,
    alpha_dom=0.1,
):
    """
    Train 1 epoch với Adversarial DA:
      L_total = L_cls + alpha_dom * L_domain
    """
    model.train()
    domain_disc.train()
    total_loss = total_cls = total_dom = 0.0
    correct, total = 0, 0
    dom_correct, dom_total = 0, 0

    for g, seq, labs, doms in loader:
        g, seq, labs, doms = g.to(device), seq.to(device), labs.to(device), doms.to(device)

        opt_main.zero_grad()
        opt_disc.zero_grad()

        # 1) Forward encoder + classifier => logits, feat
        logits, feat = model(g, seq, return_features=True)

        # Align batch nếu mismatch
        b_log, b_lab = logits.size(0), labs.size(0)
        if b_log != b_lab:
            b = min(b_log, b_lab)
            logits = logits[:b]
            labs = labs[:b]
            doms = doms[:b]
            feat = feat[:b]

        # 2) Classification loss
        cls_loss = cls_crit(logits, labs)

        # 3) Domain loss (adversarial nhờ GRL)
        dom_logits = domain_disc(feat, lambd=lambda_da)
        dom_loss = dom_crit(dom_logits, doms)

        # 4) Tổng loss
        loss = cls_loss + alpha_dom * dom_loss

        loss.backward()
        opt_main.step()
        opt_disc.step()

        # ====== Thống kê ======
        total_loss += loss.item() * labs.size(0)
        total_cls += cls_loss.item() * labs.size(0)
        total_dom += dom_loss.item() * labs.size(0)

        # acc phân loại ransomware/benign
        preds = (torch.sigmoid(logits) > 0.5).float()
        correct += (preds == labs).sum().item()
        total += labs.size(0)

        # acc domain (source/target) của D
        dom_pred = (torch.sigmoid(dom_logits) > 0.5).float()
        dom_correct += (dom_pred == doms).sum().item()
        dom_total += doms.size(0)

    return {
        "loss": total_loss / total,
        "cls_loss": total_cls / total,
        "dom_loss": total_dom / total,
        "acc": correct / total,
        "dom_acc": dom_correct / max(dom_total, 1e-9),
    }


def train_epoch(model, loader, crit, opt, device):
    """
    Train thường (không DA) – giữ lại nếu muốn chạy baseline.
    """
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for g, seq, labs in loader:
        g, seq, labs = g.to(device), seq.to(device), labs.to(device)

        opt.zero_grad()
        logits = model(g, seq)

        if logits.size(0) != labs.size(0):
            b = min(logits.size(0), labs.size(0))
            logits = logits[:b]
            labs = labs[:b]

        loss = crit(logits, labs)
        loss.backward()
        opt.step()

        total_loss += loss.item() * labs.size(0)
        preds = (torch.sigmoid(logits) > 0.5).float()
        correct += (preds == labs).sum().item()
        total += labs.size(0)

    return total_loss / total, correct / total


def eval_metrics(model, loader, crit, device):
    model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for g, seq, labs in loader:
            g, seq, labs = g.to(device), seq.to(device), labs.to(device)
            logits = model(g, seq)

            if logits.size(0) != labs.size(0):
                b = min(logits.size(0), labs.size(0))
                logits = logits[:b]
                labs = labs[:b]

            total_loss += crit(logits, labs).item() * labs.size(0)

            preds = (torch.sigmoid(logits) > 0.5).float()
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labs.cpu().tolist())

    if len(set(all_labels)) < 2:
        return {
            'loss': total_loss,
            'acc': 0, 'tpr': 0, 'fpr': 0,
            'precision': 0, 'recall': 0, 'f1': 0
        }

    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds).ravel()
    total = tp + tn + fp + fn

    return {
        'loss': total_loss / total,
        'acc': (tp + tn) / total,
        'tpr': tp / (tp + fn + 1e-9),
        'fpr': fp / (fp + tn + 1e-9),
        'precision': precision_score(all_labels, all_preds),
        'recall': recall_score(all_labels, all_preds),
        'f1': f1_score(all_labels, all_preds)
    }


# ======================================================
# PSI utilities – drift CLEAN vs NOISE (ransomware-only)
# ======================================================

def _is_ransom(ds, idx: int) -> bool:
    """
    Kiểm tra sample ở ds.samples[idx] có phải ransomware (label=1) hay không.
    """
    return int(ds.samples[idx][2]) == 1


@torch.no_grad()
def collect_model_outputs_branch(model, ds, indices, device, batch_size=32):
    """
    Thu output cho PSI từ 1 dataset:
      - prob = sigmoid(logits)
      - w_seq, w_graph (từ gating)
    """
    if len(indices) == 0:
        return {"prob": np.array([]), "w_seq": np.array([]), "w_graph": np.array([])}

    loader = DataLoader(
        Subset(ds, indices),
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn
    )

    model.eval()
    probs, wseqs, wgraphs = [], [], []

    for g, seq, _ in loader:
        g = g.to(device)
        seq = seq.to(device)

        logits, br = model(g, seq, return_branch=True)
        p = torch.sigmoid(logits).detach().cpu().numpy()

        probs.append(p)
        wseqs.append(br["w_seq"].numpy().reshape(-1))
        wgraphs.append(br["w_graph"].numpy().reshape(-1))

    return {
        "prob": np.concatenate(probs, axis=0),
        "w_seq": np.concatenate(wseqs, axis=0),
        "w_graph": np.concatenate(wgraphs, axis=0),
    }


def make_bins_from_reference(ref_values, n_bins=10, strategy="quantile"):
    ref_values = np.asarray(ref_values)
    if ref_values.size == 0:
        edges = np.linspace(0.0, 1.0, n_bins + 1)
        edges[0] = -np.inf
        edges[-1] = np.inf
        return edges

    if strategy == "fixed":
        edges = np.linspace(0.0, 1.0, n_bins + 1)
    else:
        qs = np.linspace(0.0, 1.0, n_bins + 1)
        edges = np.quantile(ref_values, qs)
        edges = np.unique(edges)
        if len(edges) < 3:
            edges = np.linspace(0.0, 1.0, n_bins + 1)

    edges[0] = -np.inf
    edges[-1] = np.inf
    return edges


def psi(expected, actual, bin_edges, eps=1e-6):
    expected = np.asarray(expected)
    actual = np.asarray(actual)

    e_counts, _ = np.histogram(expected, bins=bin_edges)
    a_counts, _ = np.histogram(actual, bins=bin_edges)

    e_perc = e_counts / max(e_counts.sum(), 1)
    a_perc = a_counts / max(a_counts.sum(), 1)

    e_perc = np.clip(e_perc, eps, None)
    a_perc = np.clip(a_perc, eps, None)

    return float(np.sum((a_perc - e_perc) * np.log(a_perc / e_perc)))


def report_year_drift_psi_ransom_only(
    model, ds,
    tr_idx, val_idx, test_idx,
    domain_labels,
    device,
    batch_size=32,
    n_bins=10,
    bin_strategy_prob="quantile",
    bin_strategy_w="fixed",
):
    """
    Concept drift (YEAR) cho ransomware:
      Ref = (train+val) ∩ domain=0(2021) ∩ y=1
      Cur = test        ∩ domain=1(2025) ∩ y=1
    """
    ref_idx = [i for i in (tr_idx + val_idx) if domain_labels[i] == 0 and _is_ransom(ds, i)]
    cur_idx = [i for i in test_idx if domain_labels[i] == 1 and _is_ransom(ds, i)]

    print("\n=========== DRIFT REPORT (PSI) [RANSOMWARE YEAR] ===========")
    print(f"[SET] ref_ran={len(ref_idx)} (train+val,2021) | cur_ran={len(cur_idx)} (test,2025)")

    if len(ref_idx) == 0 or len(cur_idx) == 0:
        print("[ERROR] Thiếu ransomware ở ref hoặc cur => không tính PSI được.")
        print("===========================================================\n")
        return None

    ref_out = collect_model_outputs_branch(model, ds, ref_idx, device, batch_size=batch_size)
    cur_out = collect_model_outputs_branch(model, ds, cur_idx, device, batch_size=batch_size)

    edges_prob = make_bins_from_reference(ref_out["prob"], n_bins=n_bins, strategy=bin_strategy_prob)
    psi_prob = psi(ref_out["prob"], cur_out["prob"], edges_prob)

    edges_w = make_bins_from_reference(ref_out["w_graph"], n_bins=n_bins, strategy=bin_strategy_w)
    psi_wg = psi(ref_out["w_graph"], cur_out["w_graph"], edges_w)
    psi_ws = psi(ref_out["w_seq"],   cur_out["w_seq"],   edges_w)

    print(f"[PSI] PSI(prob)   ransom-only: {psi_prob:.6f}")
    print(f"[PSI] PSI(w_graph) ransom-only: {psi_wg:.6f}")
    print(f"[PSI] PSI(w_seq)   ransom-only: {psi_ws:.6f}")
    print("===========================================================\n")

    return {
        "psi_prob_ransom": psi_prob,
        "psi_w_graph_ransom": psi_wg,
        "psi_w_seq_ransom": psi_ws,
        "refN_ransom": len(ref_idx),
        "curN_ransom": len(cur_idx),
    }


# ======================================================
# Split từ metadata + domain label (source/target)
# ======================================================

def build_split_indices_from_metadata(ds, metadata_csv_path):
    """
    metadata.csv:
      - filename: 'report_xxx.json'
      - split_KB1: 'train' / 'val' / 'test'
      - domain: 'source' / 'target' (hoặc pre/old/2021 vs post/new/2025...)

    Trả về:
      - train_idx, val_idx, test_idx
      - domain_labels: list cùng độ dài ds.samples, mỗi phần tử là 0 (source) hoặc 1 (target)
    """
    split_map = {}   # base_name -> (split, dom_id)

    with open(metadata_csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            fname = row['filename']            # ví dụ 'report_abc123.json'
            base = os.path.splitext(fname)[0]  # 'report_abc123'
            split = row['split_KB1'].strip().lower()

            dom_str = row.get('domain', 'source').strip().lower()
            if dom_str in ('source', 'src', 's', 'pre', 'old', '2021'):
                dom_id = 0
            elif dom_str in ('target', 'tgt', 't', 'post', 'new', '2025'):
                dom_id = 1
            else:
                dom_id = 0  # default source

            split_map[base] = (split, dom_id)

    train_idx, val_idx, test_idx = [], [], []
    domain_labels = [0] * len(ds.samples)
    missing = 0

    for idx, (g_path, seq_path, lbl) in enumerate(ds.samples):
        base_g = os.path.splitext(os.path.basename(g_path))[0]
        val = split_map.get(base_g)

        if val is None:
            missing += 1
            continue

        split, dom_id = val
        domain_labels[idx] = dom_id

        if split == "train":
            train_idx.append(idx)
        elif split == "val":
            val_idx.append(idx)
        elif split == "test":
            test_idx.append(idx)
        else:
            missing += 1

    train_idx = sorted(train_idx)
    val_idx = sorted(val_idx)
    test_idx = sorted(test_idx)

    print(f"[INFO] Split indices from metadata: "
          f"train={len(train_idx)}, val={len(val_idx)}, test={len(test_idx)}, "
          f"missing_meta={missing}")
    return train_idx, val_idx, test_idx, domain_labels


# ======================================================
# Run function (GCN + xLSTM + Adversarial DA)
# ======================================================

def run_gcn_xlstm():
    # ===== GRAPH PATHS =====
    benign_graph_dir = "/kaggle/input/2026benign-1633rans21/PYGdata_benign_2026_Rans21_1633/benign"

    # ransomware CLEAN (2021 + 2025)
    rans21_clean_graph_dir = "/kaggle/input/2026benign-1633rans21/PYGdata_benign_2026_Rans21_1633/ransomware"
    rans25_clean_graph_dir = "/kaggle/input/557rans25/pyg_data_DistilBERT_ransomware2025/ransomware"

    # ransomware NOISE (2021 + 2025)
    rans21_noise_graph_dir = "/kaggle/input/data-rans21-noise/pyg_data_DistilBERT_ransomware2021_noise/ransomware"
    rans25_noise_graph_dir = "/kaggle/input/data-rans25-noise/pyg_data_DistilBERT_ransomware2025_noise/ransomware"

    ransomware_graph_dirs_clean = [
        rans21_clean_graph_dir,
        rans25_clean_graph_dir,
    ]
    ransomware_graph_dirs_noise = [
        rans21_noise_graph_dir,
        rans25_noise_graph_dir,
    ]

    # ===== SEQ PATHS =====
    benign_seq_root = "/kaggle/input/seq-ids/seq_ids"
    ransomware_seq_root_clean = "/kaggle/input/seq-ids/seq_ids"
    ransomware_seq_root_noise = "/kaggle/input/seq-ids-noise/seq_ids_noise"

    # ===== Vocab & metadata =====
    vocab_json_path = "/kaggle/input/seq-vocab/vocab_runtime.json"
    metadata_csv_path = "/kaggle/input/csv-kb1-q/metadata_KB1_Q.csv"

    # Load vocab
    with open(vocab_json_path, "r", encoding="utf-8") as f:
        vocab = json.load(f)
    vocab_size = len(vocab)
    print(f"[INFO] Vocab size = {vocab_size}")

    seq_len_options = [1500]
    batch_size = 8
    lr = 1e-3
    epochs = 20
    patience = 5

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"[INFO] Using device: {device}")

    for seq_len in seq_len_options:
        print(f"\n=== [SEQ_LEN={seq_len}] (RETRAIN, train CLEAN, test NOISE) ===")

        # ===== Dataset CLEAN cho train/val (2021 + early 2025, không noise) =====
        try:
            ds_clean = MultiModalDatasetPT(
                benign_graph_dir,
                ransomware_graph_dirs_clean,
                benign_seq_root=benign_seq_root,
                ransomware_seq_root=ransomware_seq_root_clean,
                max_seq_len=seq_len
            )
        except RuntimeError as e:
            print(f"[WARN] {e} → skip combo này.")
            return

        print(f"[INFO][CLEAN] Found {len(ds_clean)} samples (before splitting)")

        if len(ds_clean) < 10:
            print("[WARNING] Not enough CLEAN samples. Skipping...")
            return

        # Split + domain label từ metadata cho CLEAN
        print("[INFO] Build split for CLEAN dataset (train/val, domain=2021 vs early-2025)")
        tr_idx, val_idx, test_idx_clean, domain_labels_clean = build_split_indices_from_metadata(
            ds_clean, metadata_csv_path
        )
        if min(len(tr_idx), len(val_idx)) == 0:
            print("[ERROR] train hoặc val rỗng. Kiểm tra metadata_KB1_final.csv.")
            return

        # Dataloader cho RETRAIN (không dùng domain label trong training)
        tr_loader = DataLoader(
            Subset(ds_clean, tr_idx),
            batch_size=batch_size,
            shuffle=True,
            collate_fn=collate_fn
        )
        val_loader = DataLoader(
            Subset(ds_clean, val_idx),
            batch_size=batch_size,
            shuffle=False,
            collate_fn=collate_fn
        )

        # Lấy feature dim từ sample CLEAN
        sample_g, _, _ = ds_clean[0]
        in_feats = sample_g.x.size(1)
        print(f"[INFO] Node feature dim = {in_feats}")

        # Encoders + classifier
        graph_enc = GCNEncoder(in_feats).to(device)
        seq_enc = xLSTMEncoder(vocab_size, seq_len=seq_len).to(device)
        model = MultiModalClassifier(graph_enc, seq_enc).to(device)

        cls_crit = nn.BCEWithLogitsLoss()
        opt = torch.optim.Adam(model.parameters(), lr=lr)

        best_f1 = 0.0
        no_improve = 0
        best_state = None

        # ===== Train RETRAIN (không DA) trên CLEAN (2021 + early 2025) =====
        for epoch in range(1, epochs + 1):
            tr_loss, tr_acc = train_epoch(
                model,
                tr_loader,
                crit=cls_crit,
                opt=opt,
                device=device
            )
            val_met = eval_metrics(model, val_loader, cls_crit, device)

            print(
                f"[Epoch {epoch}] "
                f"Train Loss={tr_loss:.6f}, Train Acc={tr_acc:.6f} | "
                f"Val Loss={val_met['loss']:.6f}, Val Acc={val_met['acc']:.6f}, "
                f"Val F1={val_met['f1']:.6f}"
            )

            # Early stopping theo Val F1
            if val_met['f1'] > best_f1:
                best_f1 = val_met['f1']
                best_state = model.state_dict()
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print("Early stopping.")
                    break

        # Load lại best state
        if best_state is not None:
            model.load_state_dict(best_state)

        # ===== Dataset NOISE cho TEST (chỉ test, không train) =====
        try:
            ds_noise = MultiModalDatasetPT(
                benign_graph_dir,
                ransomware_graph_dirs_noise,
                benign_seq_root=benign_seq_root,
                ransomware_seq_root=ransomware_seq_root_noise,
                max_seq_len=seq_len
            )
        except RuntimeError as e:
            print(f"[WARN][NOISE] {e} → skip noise test.")
            return

        print(f"[INFO][NOISE] Found {len(ds_noise)} samples (before splitting)")

        # Lấy index test từ metadata cho NOISE
        print("[INFO] Build split for NOISE dataset (chỉ dùng test_idx_noise)")
        _, _, test_idx_noise, _ = build_split_indices_from_metadata(
            ds_noise, metadata_csv_path
        )
        if len(test_idx_noise) == 0:
            print("[ERROR] test_idx_noise rỗng (metadata không có split_KB1='test' cho noise).")
            return

        test_loader_noise = DataLoader(
            Subset(ds_noise, test_idx_noise),
            batch_size=batch_size,
            shuffle=False,
            collate_fn=collate_fn
        )

        # Đánh giá trên 2025 NOISE
        test_met_noise = eval_metrics(model, test_loader_noise, cls_crit, device)
        print(
            f">>> TEST NOISE (2025 noise, RETRAIN) => "
            f"Loss={test_met_noise['loss']:.6f}, Acc={test_met_noise['acc']:.6f}, "
            f"TPR={test_met_noise['tpr']:.6f}, FPR={test_met_noise['fpr']:.6f}, "
            f"Precision={test_met_noise['precision']:.6f}, "
            f"Recall={test_met_noise['recall']:.6f}, "
            f"F1={test_met_noise['f1']:.6f}"
        )

        # ===== PSI DRIFT: YEAR (2021 vs 2025, CLEAN ransomware-only) =====
        drift_stats = report_year_drift_psi_ransom_only(
            model, ds_clean,
            tr_idx, val_idx, test_idx_clean,
            domain_labels_clean,
            device=device,
            batch_size=batch_size,
            n_bins=10,
            bin_strategy_prob="quantile",
            bin_strategy_w="fixed",
        )
        print("[DRIFT_STATS]", drift_stats)


if __name__ == '__main__':
    set_seed(42)
    run_gcn_xlstm()


[INFO] Vocab size = 4509056
[INFO] Using device: cuda

=== [SEQ_LEN=1500] (RETRAIN, train CLEAN, test NOISE) ===
[INFO] Tổng số sample: 4215
[INFO][CLEAN] Found 4215 samples (before splitting)
[INFO] Build split for CLEAN dataset (train/val, domain=2021 vs early-2025)
[INFO] Split indices from metadata: train=3020, val=521, test=651, missing_meta=23
[INFO] Node feature dim = 768
[Epoch 1] Train Loss=0.226061, Train Acc=0.901325 | Val Loss=0.168185, Val Acc=0.980806, Val F1=0.984375
[Epoch 2] Train Loss=0.117203, Train Acc=0.943709 | Val Loss=0.226202, Val Acc=0.982726, Val F1=0.985959
[Epoch 3] Train Loss=0.101480, Train Acc=0.949338 | Val Loss=0.285060, Val Acc=0.980806, Val F1=0.984375
[Epoch 4] Train Loss=0.096630, Train Acc=0.951987 | Val Loss=0.274868, Val Acc=0.976967, Val F1=0.981191
[Epoch 5] Train Loss=0.090049, Train Acc=0.962252 | Val Loss=0.188720, Val Acc=0.986564, Val F1=0.989114
[Epoch 6] Train Loss=0.048711, Train Acc=0.983775 | Val Loss=0.260123, Val Acc=0.978887, Val 